In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

#### Filepaths and Filelist

In [22]:
path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/all/nc/*.nc'
flist = glob.glob(path)

#### Create and Close XarrayDataset

In [23]:
ds=xr.open_mfdataset(flist, concat_dim='time')
ds.close()

#### Specs. on Dataset (ds)

In [24]:
# few others....but these are main
ds.dims
ds.values
ds.coords
ds.Band1.shape

(51, 16781, 17568)

#### Assign time coordinate baised on lidar flight date & select time by year (or specific date)

In [25]:
date=[]

for f in sorted(flist):
#     print(f)
    dt_str = f.split("/")[-1] #splits on / and saves the last one
    dt_str = "".join([c for c in dt_str if c.isnumeric()]) #grabs numeric values
    dt = pd.to_datetime(dt_str[:8]) #convert to datetime, grabs digits for dates only
#     print('file:',dt.date())
    date.append(dt.date())

time_var = pd.to_datetime(date)
ds['DATE'] = (['time'], time_var)
ds = ds.assign_coords(time=ds.DATE)

ds


<xarray.Dataset>
Dimensions:              (time: 51, x: 17568, y: 16781)
Coordinates:
  * x                    (x) float64 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y                    (y) float64 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
  * time                 (time) datetime64[ns] 2013-04-03 ... 2018-05-28
Data variables:
    transverse_mercator  (time) |S1 b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    Band1                (time, y, x) float64 dask.array<shape=(51, 16781, 17568), chunksize=(1, 16781, 17568)>
    DATE                 (time) datetime64[ns] 2013-04-03 ... 2018-05-28
Attributes:
    GDAL_AREA_OR_POINT:     Area
    GDAL_TIFFTAG_SOFTWARE:  MATLAB 8.5, Mapping Toolbox 4.1
    Conventions:            CF-1.5
    GDAL:                   GDAL 2.2.3, released 2017/11/20
    history:                Thu Aug 01 17:17:04 2019: GDAL CreateCopy( /home/...

In [ ]:
#SELECT YEAR, or DATE
# ds = ds.sel(time='2015')
ds = ds.sel(time='2018')

In [32]:
dc = ds['DATE'].chunks
d = ds.chunks

print(d)

Frozen(SortedKeysDict({'time': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), 'y': (16781,), 'x': (17568,)}))


In [26]:
d=ds['Band1'].chunks #dataArray
d

TypeError: 'method' object is not subscriptable

#### compute stuff (testing)

In [8]:
# variance in snow depth
v = ds.Band1.var(dim='time') #compute variance on every pixel in the time dimention 
v = np.where(v>.001, v, np.nan) # filters everything greater than 1mm (i.e gives alpha bg)
print(v)










TypeError: '>' not supported between instances of 'tuple' and 'float'

In [6]:
# t=time.time()

# variance in snow depth
varsn = ds.Band1.var(dim='time') #compute variance on every pixel in the time dimention 
varsn = np.where(varsn>.001, varsn, np.nan) # filters everything greater than 1mm (i.e gives alpha bg)
print(varsn)
# fig = plt.figure(figsize=(15, 10))
# plt.imshow(varsn, cmap='jet', origin='lower', vmin=.05, vmax=1.0) #[::-1,:] - reverses it
# plt.title('snow depth [3m] variance, 2018')
# plt.colorbar()
# plt.savefig('figs/var_13-16', dpi=600)

# tt=time.time()
# print('runtime =', (tt-t)/60)

MemoryError: 